- Calificación de IMDB.
- Dirección (director o directores).
- Guionistas.
- Argumento.
- Duración (en minutos).

In [40]:
import pandas as pd
from tqdm import tqdm

import pickle 
import sys
sys.path.append('../')

In [41]:
from bs4 import BeautifulSoup
import requests

In [42]:
with open(f'../data/full_data.pkl', 'rb') as f:
    data = pickle.load(f)

In [43]:
df = pd.DataFrame(data, columns=['type', 'title', 'year', 'month', 'id', 'genre'])

In [44]:
df['year'].unique()

array([2024.,   nan, 2025., 2023., 2022.])

Vemos que realmente solo tenemos información de películas de 2022 a 2025. Esto indica que tendríamos que revisar que la extracción de información de la API funciona correctamente en el paso anterior.

De momento vamos a continuar con esta información, y si nos sobra tiempo lo hacemos.

In [45]:
df.duplicated().sum()

0

No hay duplicados, así que bien. Pero...

In [46]:
df['id'].duplicated().sum()

1115

Resulta que hay duplicados porque algunas películas tienen varios géneros. Por el momento lo vamos a dejar así y usaremos el índice como identificador de la entrada

In [47]:
df[df['id'].isin(df[df['id'].duplicated()]['id'].unique())].sort_values(by='id').head(10)

,type,title,year,month,id,genre
0,movie,Mom's Coming,2024.0,11.0,tt10579986,Drama
4000,movie,Mom's Coming,2024.0,11.0,tt10579986,Horror
7000,movie,Mom's Coming,2024.0,11.0,tt10579986,Thriller
2000,movie,Slasher House 3,2024.0,3.0,tt11078340,Action
4001,movie,Slasher House 3,2024.0,3.0,tt11078340,Horror
3000,movie,Amigos imaginarios,2024.0,5.0,tt11152168,Fantasy
1001,movie,Amigos imaginarios,2024.0,5.0,tt11152168,Comedy
3,movie,Amigos imaginarios,2024.0,5.0,tt11152168,Drama
5214,movie,Vincent,2023.0,NaN,tt11253182,Mystery
3220,movie,Vincent,2023.0,NaN,tt11253182,Fantasy


Para que sea viable un scrapeo en poco tiempo vamos a utilizar solamente algunas películas. 

Comencemos utilizando solo las de 2024 que sean drama y no sean cortos, solo largometrajes.

In [48]:
filtro = (df['year'] == 2024) & (df['genre'] == 'Drama') & (df['type'] == 'movie')

df_filtrado = df[filtro]

df_filtrado.shape[0]

489

De hecho comprobamos que no haya duplicados ahora.

In [49]:
df_filtrado['id'].duplicated().sum()

0

In [75]:
df_filtrado.to_csv('../data/datos_api.csv')

Tenemos 489 películas, vayamos con esto por el momento

In [50]:
def get_synopsis(id_):

    url = f'https://www.imdb.com/title/{id_}/plotsummary'

    # Simulamos un usuario
    headers = {'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_11_5) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/50.0.2661.102 Safari/537.36'}
    response = requests.get(url, headers=headers)

    # Comprobamos que obtenemos un 200
    if response.status_code != 200:
        print("Error:", response.status_code)
        print("id:", id_)
        return

    soup = BeautifulSoup(response.content, 'html.parser')

    sinopsis = soup.find("div", {"data-testid": "sub-section-synopsis"}).text

    return sinopsis

In [51]:
def scrap_id(id_):

    url = f'https://www.imdb.com/es-es/title/{id_}/'

    # Simulamos un usuario
    headers = {'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_11_5) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/50.0.2661.102 Safari/537.36'}
    response = requests.get(url, headers=headers)

    # Comprobamos que obtenemos un 200
    if response.status_code != 200:
        print("Error:", response.status_code)
        print("id:", id_)
        return

    # Let him cook (the soup)
    soup = BeautifulSoup(response.content, 'html.parser')

    # Puntuación
    try:
        score_ = soup.find("div", {'data-testid': "hero-rating-bar__aggregate-rating__score"}).text
        score_ = float(score_.split('/')[0].replace(',', '.'))
    except:
        score_ = None

    # Dirección, Guión, Reparto Principal
    info = soup.find("ul", {'class':"ipc-metadata-list ipc-metadata-list--dividers-all title-pc-list ipc-metadata-list--baseAlt"})
    info_table = info.findAll("li", {"class": "ipc-inline-list__item"})

    directores_ = []
    guionistas_ = []
    actores_ = []

    for item in info_table:

        class_ = item.a.get('href').split('=')[1]

        if 'dr' in class_:
            directores_.append(item.text)

        elif 'wr' in class_:
            guionistas_.append(item.text)

        elif 'st' in class_:
            actores_.append(item.text)

    # Resumen
    try:
        resumen_ = soup.find("span", {'data-testid':"plot-l"}).text
    except:
        resumen_ = None

    # Duración
    try:
        duracion_ = soup.find("ul", {"class":"ipc-inline-list ipc-inline-list--show-dividers sc-ec65ba05-2 joVhBE baseAlt"}).findAll("li", {'class': "ipc-inline-list__item"})[-1].text
        # Si dura menos de 1h no aparece h
        if 'h' in duracion_:
            horas, minutos = duracion_.split(' ')
            duracion_ = int(horas.rstrip('h').strip())*60 + int(minutos.rstrip('min').strip())
        else:
            duracion_ = int(minutos.rstrip('min').strip())


    except:
        duracion_ = None

    # Sinopsis
    try:
        sinopsis_ = get_synopsis(id_)

    except:
        sinopsis_ = None

    return (id_, score_, directores_, guionistas_, actores_, resumen_, sinopsis_, duracion_)

In [52]:
datos = []

for id_ in tqdm(df_filtrado['id'].values):
    datos.append(scrap_id(id_))

  3%|▎         | 15/489 [00:30<12:20,  1.56s/it]

Error: 404
id: tt14643262


  4%|▍         | 20/489 [00:40<15:00,  1.92s/it]

Error: 404
id: tt15535164


  4%|▍         | 22/489 [00:43<13:01,  1.67s/it]

...

 98%|█████████▊| 478/489 [22:01<00:28,  2.55s/it]

Error: 404
id: tt31170218


 99%|█████████▉| 486/489 [22:22<00:06,  2.32s/it]

Error: 404
id: tt31172843


100%|██████████| 489/489 [22:30<00:00,  2.76s/it]


In [54]:
df_scrap = pd.DataFrame(datos, columns=['id', 'score', 'directores', 'guionistas', 'actores', 'resumen', 'sinopsis', 'duracion'])

Eliminamos las filas en las que todos los valores sean nulos

In [60]:
df_scrap.dropna(how='all', inplace=True)

In [74]:
df_scrap.to_csv('../data/datos_scrap.csv')

In [76]:
actor_list = []

for entry in df_scrap['actores'].to_list():
    for actor in entry:
        actor_list.append(actor)

In [85]:
len(set(actor_list))

1284

Es inviable ahora mismo que hagamos una búsqueda de los 1284 actores únicos, principalmente porque esta parte sí va a requerir usar Selenium. Vamos a contar cuáles aparecen más de una vez y a quedarnos con esos por el momento.

In [86]:
from collections import Counter
cnt = Counter(actor_list)

In [89]:
df_actors = pd.DataFrame(cnt.most_common(), columns=['actor', 'count'])

In [91]:
df_actors[df_actors['count'] > 1].shape[0]

31

Vemos que de esta forma tenemos 31 actores, vamos a tirar con esto.

In [94]:
actores_repetidos = df_actors[df_actors['count'] > 1]['actor'].to_list()

In [95]:
actores_repetidos

['Charity Lovely Badazz Alston',
 'Mike Ajide',
 'Karen Abercrombie',
 'Jane John-Nwankwo',
 'Elaine Aikens',
 'Richard Brown Jr.',
 'Raashi Khanna',
 'Aglaya Tarasova',
 'Sidse Babett Knudsen',
 'Chelle Alexis',
 'Rese Cheri',
 'Bing Jia',
 'Brandon Bell',
 'Rayana Briggs',
 'Micheal Ball',
 'Rob Fury',
 'John U. Nwankwo',
 'Yuriy Tsurilo',
 'Towanda Braxton',
 'Denzell Dandridge',
 'Crystal The Doll',
 'Tasnia Farin',
 'Ishrat Zaheen Ahmed',
 'Shahed Ali',
 'Nick Launchbury',
 'Divya Gowda',
 'Isabelle Huppert',
 'Shishir Ahmed',
 'Tuhin Chowdhury',
 'Donny Damara',
 'Egypt Bradley']

In [104]:
df_filtrado.head()

,type,title,year,month,id,genre
0,movie,Mom's Coming,2024.0,11.0,tt10579986,Drama
1,movie,Love Song and Power,2024.0,1.0,tt10597228,Drama
2,movie,Spaceman,2024.0,3.0,tt11097384,Drama
3,movie,Amigos imaginarios,2024.0,5.0,tt11152168,Drama
4,movie,Manje Bistre 3,2024.0,7.0,tt11822244,Drama


In [107]:
df_filtrado.shape[0]

489

In [105]:
df_scrap.head()

,id,score,directores,guionistas,actores,resumen,sinopsis,duracion
0,tt10579986,NaN,[Rishiking],[Rishiking],[Byom],Un alma supuestamente tiene como hija a una ch...,None,122.0
1,tt10597228,NaN,"[Erik Krefeld, Eddel Martinez]","[Stevie Kinchen, Erik Krefeld]","[Aaron Jackson, Rose LaRue, Cassidy Reyes]",Esta es una historia sobre encontrar el valor ...,None,90.0
2,tt11097384,5.7,[Johan Renck],"[Jaroslav Kalfar, Colby Day]","[Adam Sandler, Carey Mulligan, Paul Dano]","Jakub Procházka, huérfano y criado en el campo...",Czech astronaut Jakub Prochazka (Adam Sandler)...,107.0
3,tt11152168,6.4,[John Krasinski],[John Krasinski],"[Cailey Fleming, Ryan Reynolds, John Krasinski]",Sigue a una niña que pasa por una experiencia ...,A 12 year old girl named Bea moves in with her...,104.0
4,tt11822244,8.8,[Baljit Singh Deo],[Rana Ranbir],"[Karamjit Anmol, Raghveer Boli, Raaf Chaudhary]",Revolves around the Punjabi wedding culture.,None,NaN


In [113]:
df_final = df_filtrado.merge(right=df_scrap, left_on='id', right_on='id')

In [115]:
df_final.to_csv('../data/datos_final.csv')